In [1]:
#__bojo90__

from __future__ import print_function
from fenics import *
import mshr
import numpy as np

%matplotlib inline

######################################################################################################################
class ChemicalReactionrate:
      
    def __init__(self, atmosphericTemp=300, cloudCover=0.5, solarElevation=0.79):
        self.atmosphericTemp = atmosphericTemp
        self.cloudCover = cloudCover
        self.solarElevation = solarElevation
        
        
    def reactionRate(self):
        """Defining the chemical reaction coefficient """ 
        k1 = 1/60*(0.5699-(9.056*(10**3)*(90-self.solarElevation))**2.546)*(1-0.75*(self.cloudCover/8)**3.4)
        k2 = 1.325*10**6*np.exp(-1430/self.atmosphericTemp)
        return k2/k1      


######################################################################################################################
class Domain:
    
    grid = mshr.Rectangle(Point(0, 0), Point(2.5, 2.5))
    mesh = mshr.generate_mesh(grid, 64)
    
    # Define function space for velocity
    velocityFunctionSpace = VectorFunctionSpace(mesh, 'P', 2)

    # Define function space for system of concentrations
    P1 = FiniteElement('P', triangle, 1)
    element = MixedElement(P1, P1, P1, P1)
    concentrationFunctionSpace = FunctionSpace(mesh, element)
    
    def velocityFunction(self):
        # Define functions for velocity
        velocityFunction = Function(Domain.velocityFunctionSpace)
        return velocityFunction
        
    def concentrationFunction(self):
        ''' Define functions for Concentrations'''         
        concentrationFunction = Function(Domain.concentrationFunctionSpace)
        return concentrationFunction  
domain = Domain()
concentrationfunctioninitial = domain.concentrationFunction()
concentrationfunctionfinal = domain.concentrationFunction()
velocityfunction = domain.velocityFunction()

        
#####################################################################################################################    
class PlumeSource:
    # Define source terms
    def NO_Source(self):
        self.NO_Source = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.2,2)<0.05*0.05 ? 0.15 : 0', degree=1)
        return self.NO_Source
        
    def O_3_Source(self):
        self.O_3_Source = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.3,2)<0.05*0.05 ? 0.15 : 0', degree=1)
        return self.O_3_Source
        
    def NO_2_Source(self):
        self.NO_2_Source = Constant(0)
        return self.NO_2_Source

    def SO_2_Source(self):
        self.SO_2_Source = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.4,2)<0.05*0.05 ? 0.07 : 0', degree=1)
        return self.SO_2_Source

       
#####################################################################################################################    
class VariationalSolver(PlumeSource):

    totalTime = 5.0            # final time
    num_steps = 500    # number of time steps
    timestep = totalTime / num_steps # time step size
    eps = 0.025         # diffusion coefficient

    ChemicalReaction = ChemicalReactionrate()
    reactionrate = ChemicalReaction.reactionRate()
    
    # Define expressions used in variational forms
    reactionrate = Constant(reactionrate)
    timeStep = Constant(timestep)
    eps = Constant(eps) 

    def solver(self):
        concentrationSplitInitialTime = split(concentrationfunctioninitial)
        concentrationSplitFinalTime = split(concentrationfunctionfinal)
        NO_testfunction, O_3_testfunction, NO_2_testfunction,  SO_2_testfunction = TestFunctions(
                                                                                domain.concentrationFunctionSpace)
        NO_initial, O_3_initial, NO_2_initial, SO_2_initial = concentrationSplitInitialTime      
        NO_final, O_3_final, NO_2_final, SO_2_final = concentrationSplitFinalTime        
        solver = ((NO_initial - NO_final) / VariationalSolver.timeStep)*NO_testfunction*dx \
                + dot(velocityfunction, grad(NO_initial))*NO_testfunction*dx \
                + VariationalSolver.eps*dot(grad(NO_initial), grad(NO_testfunction))*dx \
                + VariationalSolver.reactionrate*NO_initial*O_3_initial*NO_testfunction*dx \
                + ((O_3_initial - O_3_final) / VariationalSolver.timeStep)*O_3_testfunction*dx \
                + dot(velocityfunction, grad(O_3_initial))*O_3_testfunction*dx \
                + VariationalSolver.eps*dot(grad(O_3_initial), grad(O_3_testfunction))*dx \
                + VariationalSolver.reactionrate*NO_initial*O_3_initial*O_3_testfunction*dx \
                + ((NO_2_initial - NO_2_final) / VariationalSolver.timeStep)*NO_2_testfunction*dx \
                + dot(velocityfunction, grad(NO_2_initial))*NO_2_testfunction*dx \
                + VariationalSolver.eps*dot(grad(NO_2_initial), grad(NO_2_testfunction))*dx \
                - VariationalSolver.reactionrate*NO_initial*O_3_initial*NO_2_testfunction*dx \
                + VariationalSolver.reactionrate*NO_2_initial*NO_2_testfunction*dx \
                +((SO_2_initial - SO_2_final) / VariationalSolver.timeStep)*SO_2_testfunction*dx \
                + dot(velocityfunction, grad(SO_2_initial))*SO_2_testfunction*dx \
                + VariationalSolver.eps*dot(grad(SO_2_initial), grad(SO_2_testfunction))*dx \
                - self.NO_Source()*NO_testfunction*dx - self.O_3_Source()*O_3_testfunction*dx \
                - self.NO_2_Source()*NO_2_testfunction*dx - self.SO_2_Source()*SO_2_testfunction*dx

       
        return solver

######################################################################################################################    
def run_solver():   
      
    variationalsolver = VariationalSolver()
    solver = variationalsolver.solver()
    num_steps = variationalsolver.num_steps
    timesteP = variationalsolver.timestep


    # Create time series for reading velocity data
    timeseries_velocityfunction = TimeSeries('wind-data/wind_data22')

    # Create VTK files for visualization output
    vtkfile_NO = File('reaction_system/NO.pvd')
    vtkfile_O_3 = File('reaction_system/O_3.pvd')
    vtkfile_NO_2 = File('reaction_system/NO_2.pvd')
    vtkfile_SO_2 = File('reaction_system/SO_2.pvd')

    # Time-stepping
    t = 0
    for n in range(num_steps):

        # Update current time
        t += timesteP

        # Read velocity from file
        timeseries_velocityfunction.retrieve(velocityfunction.vector(), t)

        # Solve variational problem for time step
        solve(solver == 0, concentrationfunctioninitial)
       
        # Save solution to file (VTK)
        _NO, _O_3, _NO_2, _SO_2 = concentrationfunctioninitial.split()
        vtkfile_NO << (_NO, t)
        vtkfile_O_3 << (_O_3, t)
        vtkfile_NO_2 << (_NO_2, t)
        vtkfile_SO_2 << (_SO_2, t)

        # Update previous solution
        concentrationfunctionfinal.assign(concentrationfunctioninitial)
        
if __name__ == '__main__':
    run_solver()

 

RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to open file to retrieve series.
*** Reason:  File does not exist.
*** Where:   This error was encountered inside TimeSeries.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2018.1.0
*** Git changeset:  unknown
*** -------------------------------------------------------------------------


In [2]:
import numpy as np
np.shape(velocityfunction.vector())

(51334,)

In [ ]:
import numpy as np
class ChemicalReactionrate:
      
    def __init__(self, atmosphericTemp=300, cloudCover=0.5, solarElevation=0.79):
        self.atmosphericTemp = atmosphericTemp
        self.cloudCover = cloudCover
        self.solarElevation = solarElevation
        
        
    def reactionRate(self):
        """Defining the chemical reaction coefficient """ 
        k1 = 1/60*(0.5699-(9.056*(10**3)*(90-self.solarElevation))**2.546)*(1-0.75*(self.cloudCover/8)**3.4)
        k2 = 1.325*10**6*np.exp(-1430/self.atmosphericTemp)
        self.reactionRate = k2/k1
        return self.reactionRate 
    
    def jk(self):
        self.jk1 = self.reactionRate *1000
        return self.jk1
       
class Trial(ChemicalReactionrate):
    
    def mask(self):
        mask = self.jk()
        return mask()
        
        print(mask())

    def test(self):
        self.test = self.jk()*1000
#         self.test = 10 + self.jk()
        return self.test
        
ChemicalReaction = ChemicalReactionrate()
reactionrate = ChemicalReaction.reactionRate()
#print(reactionrate)    
# JK = ChemicalReaction.jk()
# print(JK)


trial = Trial()
rr = trial.reactionRate()
ty = trial.jk()
yy = trial.test()
# print(ty)
print(yy)


In [ ]:
from fenics import *



class rr:

    def ffi(self):

        foo = 23
        top = Constant(foo)
        return top
    
    def tt(self):
        return self.ffi()
#         print(self.ffi)

In [ ]:
# rr().ffi()
rr().tt()